# Import

In [1]:
from anytree import Node

In [40]:
from tqdm import tqdm

In [2]:
with open('3.txt') as f:
    lines = f.readlines()

# Description of the data

In [3]:
len(lines)

20005

In [4]:
lines[0] #the number of vertices in the phenotype tree

'100000\n'

In [5]:
n = int(lines[0].rstrip())
n

100000

In [6]:
len(lines[1]) #the parent identifiers for the vertices

581244

In [7]:
parents = [int(i) for i in lines[1].rstrip().split(' ')]

In [8]:
parents[:5]

[1, 2, 3, 4, 5]

In [9]:
len(lines[2]) #information content values of the corresponding vertices

855632

In [10]:
info = [int(i) for i in lines[2].rstrip().split(' ')]

In [11]:
info[:5]

[784, 1499, 2436, 3329, 4162]

In [12]:
lines[3] #the number of diseases

'10000\n'

In [13]:
m = int(lines[3].rstrip())
m

10000

In [14]:
len(lines[4:m+4]) #contain descriptions of diseases

10000

In [17]:
lines[4],lines[m+4]

('6 80127 45276 14858 88937 85190 72807\n', '10000\n')

In [18]:
desc_d = []
for line in lines[4:m+4]:
    desc_d.append([int(i) for i in line.rstrip().split(' ')])

In [19]:
desc_d[-1]

[1, 8591]

In [20]:
lines[m+4] #the number of patients

'10000\n'

In [21]:
l = int(lines[m+4].rstrip())
l

10000

In [22]:
lines[m+4+1]

'6 49032 67424 45587 27267 5181 51174\n'

In [23]:
len(lines[m+4+1:m+4+1+l]) #descriptions of patients

10000

In [24]:
desc_p = []
for line in lines[m+4+1:m+4+1+l]:
    desc_p.append([int(i) for i in line.rstrip().split(' ')])

In [25]:
desc_p[-1]

[3, 84123, 83900, 61365]

In [26]:
lines[m+4+l]

'3 84123 83900 61365\n'

# Create Tree Structure

## Initiate root

In [27]:
root = Node(1)

In [28]:
nodes = {}
nodes[root.name] = root

## Create Tree

In [42]:
%%time

for i, j in tqdm(zip(range(2,n+1), parents)):
    name = i
    nodes[name] = Node(name, parent=nodes[j])

99999it [23:59, 69.45it/s] 

CPU times: user 23min 55s, sys: 11.1 s, total: 24min 6s
Wall time: 23min 59s


In [43]:
import gc
gc.collect()

51

# Task

## Function to find LCA

In [44]:
def commonancestors(a, b):
    if a.parent == b:
        return b
    elif b.parent == a:
        return a
    ancestors = [node.ancestors for node in [a,b]]
    common = []
    for parentnodes in zip(*ancestors): 
        parentnode = parentnodes[0]
        if all([parentnode is p for p in parentnodes[1:]]):
            common.append(parentnode)
        else:
            break
    return tuple(common)

## IC dictionary

In [45]:
# create info dictionary
infod = {}

In [46]:
for i , j in enumerate(info):
    infod[i+1] = j

# Other variables

In [59]:
# disease score
d_scores = []

In [60]:
# result output
result = []

In [61]:
# create score
score = 0

In [62]:
# create minimum
maxim = 0

# Main code block

In [63]:
%%time 

# loop through patients
for i in tqdm(range(1,l+1)):
    d_scores = []
    # loop through diseases
    for j in range(1, m+1):
        score = 0
        # loop through patient descriptions
        # skip first item, which is no. of desc
        for d in desc_p[i-1][1:]:
            maxim = 0
            # loop through disease descriptions:
            # skip first item, same reason as above
            for q in desc_d[j-1][1:]:
                # find LCA(q,d)
                anc = commonancestors(nodes[q],nodes[d])
                # anytree LCA function tweak
                try:
                    if len(anc) > 1:
                        key = anc[-1].name
                    elif len(anc) == 1:
                        key = anc[0].name
                except TypeError as e:
                    key = anc.name                        
                info_ = infod[key]
                # find max(IC(LCA(q,d)))
                if info_ >= maxim:
                    maxim = info_
            score += maxim
        d_scores.append(score)
    disease = d_scores.index(max(d_scores)) +1
    result.append(str(disease) + '\n')

  0%|          | 0/10000 [01:05<?, ?it/s]


KeyboardInterrupt: 

In [43]:
with open('result3.txt', 'w') as f:
    f.writelines(result)

In [52]:
len(result)

0

# **Too Slow, use Networkx Package in Python instead of AnyTree**